In [1]:
# file specific libraries
import torch
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import math, random, torch, collections, time, torch.nn.functional as F, networkx as nx, matplotlib.pyplot as plt, numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Linear
from torch_geometric.nn import GCNConv
from IPython.display import clear_output
from torch_geometric.utils import to_networkx
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from functools import wraps

%matplotlib inline
import sys,os
from models import *
from train_utils import *
from influence_metrics import *


In [2]:
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
import torch
import torch_geometric as tg
import pandas as pd

dataset_name = 'Cora'
dataset = Planetoid(root='../data/Planetoid', name=dataset_name, transform=NormalizeFeatures())
###
print()
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

data = dataset[0]  # Get the first graph object.

print()
print(data)
print('===========================================================================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Contains isolated nodes: {data.has_isolated_nodes()}')
print(f'Contains self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')



Dataset: Cora():
Number of graphs: 1
Number of features: 1433
Number of classes: 7

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Number of nodes: 2708
Number of edges: 10556
Average node degree: 3.90
Number of training nodes: 140
Training node label rate: 0.05
Contains isolated nodes: False
Contains self-loops: False
Is undirected: True


In [3]:
#### Simple model
## for GraphSAGE, alpha = 0.5, beta =1 
## for GAT, alpha = , beta = ?

model =GNN(input_dim = data.num_features, hidden_dim=252,
           output_dim = dataset.num_classes, n_layers=2,
           activation ='relu', slope=.1,
           device ='cpu',
          alpha = 0.5, beta = 1, alpha_res = 0, 
           gnn_type = 'symmetric') 
           
criterion = torch.nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4) #didn't include weight decay
train_acc_list, test_acc_list, loss_list, misclassified, predictions = train(150, model, criterion, optimizer, 
                                                 x= data.x, edge_index= data.edge_index, y=data.y, indicate=True,
                                                m=mask(data.train_mask, data.test_mask),
                                                scatter_size=30, plotting=False)

Training the model
Final test accuracy: 0.79
output:  tensor([3, 4, 4,  ..., 3, 3, 3])


1. run the same epoch size several time 
2. see heterogeneity or not 

In [ ]:
result = repeat_pipelines(model, dataset, data, epoch_size=100, display=True, indicate=True)

---- test number 1 ----


In [ ]:
find_common_nodes(result, [])

In [ ]:
similarities = node_similarity(dataset, nodes)

In [ ]:
for i in range(len(dataframe.columns)): 
    max_sim = 0
    idx1, idx2 = 1, 2
    for j in range(i, len(dataframe.columns)):
        similarity = dataframe[i][j] 
        if similarity != 1 and similarity != 0: 
            if max_sim < similarity: 
                max_sim = similarity 
                print(max_sim)
                idx1, idx2 = i, j
            print(f"{i}, {j} pair with similarity level {similarity:.3f}")
    
print(f"most similar pair: {idx1}, {idx2}" )

In [ ]:
dataframe

for i in 

In [ ]:
dc = compute_centrality(dataset, nodes, 'degree_centrality')
bc = compute_centrality(dataset, nodes, 'betweenness_centrality')
print(dc, bc)

# to do 
**misclassification** 
1. for each misclassified nodes, can I find a node responsible to the misclassification? 
2. what is the most dominant misclassified class? 

**influential nodes** 
1. for each influential node, can I figure out which class the node is in, and for which nodes the node results in misclassification? 
2. what is the relationship of the influential node to the misclassified node? 
3. what is the relationship amongst the influential nodes? 
4. if I take all of the influential nodes out, does that lead to increase in inaccuracy? or decrease? is there a consistent result? 


numbers are just numbers without context. try to write the print function wisely. 
compute the simrank for the entire graph - the group's simrank is significantly different from the group 
random paris of the graph and average out? 

# links
node similarity: https://towardsdatascience.com/simrank-similarity-analysis-1d8d5a18766a
 0 if have the same parents, return a similarity value


In [ ]:
outputs, output_differences, kl_scores, accuracies = check_pipeline(model, dataset, data, data.train_mask,
                   data.test_mask,
                   n_epochs=200,   
                   original_output=None,
                   indicate=True, \
                   return_prediction=True,
                   compute_y_differences=True,
                   dimension=32,
                   task='classification',
                   loss_function=torch.nn.CrossEntropyLoss(),
                   lr=0.001)

## spare codes

In [ ]:
# u = model(data.x, data.edge_index)

In [ ]:
#  F.mse_loss(u, u)

In [ ]:
# loo_pipeline(model, dataset, data,data.train_mask,
#                  data.test_mask, 1,
#                  10,
#                  original_output=None,
#                  compute_y_differences=True,
#                  task='classfication',
#                  loss_function=torch.nn.CrossEntropyLoss(),
#                  lr=0.001)